In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=api_key
)

d:\Generative_ai_practise\Basic_model_setup\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [ ]:
# tool create
#add type hints
@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [4]:
print(multiply.invoke({'a':3, 'b':4}))

12


In [5]:
multiply.name

'multiply'

In [6]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [ ]:
# tool binding

In [7]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=api_key
)

In [8]:
llm.invoke('hi')

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--42e6b411-bd14-45ba-8f22-b83e61da7843-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [9]:
llm_with_tools = llm.bind_tools([multiply])

In [10]:
llm_with_tools.invoke('Hi how are you')

AIMessage(content="I'm doing well, thank you for asking! How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--67472e5e-b90c-4cae-813f-bdc1f9b693fe-0', usage_metadata={'input_tokens': 24, 'output_tokens': 19, 'total_tokens': 43, 'input_token_details': {'cache_read': 0}})

In [11]:
query = HumanMessage('can you multiply 3 with 1000')

In [12]:
messages = [query]

In [13]:
result = llm_with_tools.invoke(messages)

In [14]:
messages.append(result)

In [15]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 1000.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--ca0b6fda-495f-4af3-aea4-a2bc19e4cc26-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 1000.0}, 'id': 'e3abc779-d4b3-4f0b-a7bd-63e803c1110a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 31, 'output_tokens': 5, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}})]

In [16]:
tool_result = multiply.invoke(result.tool_calls[0])

In [17]:
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='e3abc779-d4b3-4f0b-a7bd-63e803c1110a')

In [18]:
messages.append(tool_result)

In [19]:
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 3.0, "b": 1000.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--ca0b6fda-495f-4af3-aea4-a2bc19e4cc26-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3.0, 'b': 1000.0}, 'id': 'e3abc779-d4b3-4f0b-a7bd-63e803c1110a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 31, 'output_tokens': 5, 'total_tokens': 36, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='3000', name='multiply', tool_call_id='e3abc779-d4b3-4f0b-a7bd-63e803c1110a')]

In [20]:
llm_with_tools.invoke(messages).content

'3 multiplied by 1000 is 3000.'

In [22]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [23]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [24]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1751068801,
 'time_last_update_utc': 'Sat, 28 Jun 2025 00:00:01 +0000',
 'time_next_update_unix': 1751155201,
 'time_next_update_utc': 'Sun, 29 Jun 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.5582}

In [25]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [26]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=api_key
)

In [27]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [28]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [29]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [30]:
ai_message = llm_with_tools.invoke(messages)

In [31]:
messages.append(ai_message)

In [32]:
ai_message.tool_calls

[]

In [33]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [34]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='I need more information.  The available function `get_conversion_factor` requires a `base_currency` and a `target_currency` to be specified.  I also need the conversion rate information which is not provided in the context.  Please provide the conversion rate between INR and USD.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--ecae6cce-0129-4da9-9a9b-530ddaec6717-0', usage_metadata={'input_tokens': 92, 'output_tokens': 62, 'total_tokens': 154, 'input_token_details': {'cache_read': 0}})]

In [35]:
llm_with_tools.invoke(messages).content

'Once you provide the conversion rate, I can calculate the conversion.'

In [36]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

C:\Users\btirk\AppData\Local\Temp\ipykernel_2880\627313024.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_executor = initialize_agent(


In [37]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
Thought:I am doing well, thank you for asking.  I can help you with currency conversions or other tasks if you have any.
Action:
```json
{
  "action": "Final Answer",
  "action_input": "I'm doing well, thank you for asking! How can I help you today?"
}
```

> Finished chain.
